# [OpenAI Cookbook 예제](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb) 참조

In [2]:
import pandas as pd
import tiktoken
import openai
from openai import OpenAI
from scipy.spatial.distance import cosine
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(".env"))
client = OpenAI()
from pprint import pprint
import json
openai.api_key = os.environ["OPENAI_API_KEY"]

### 윤치호 데이터 불러오기

In [35]:
df = pd.read_csv("thy_diary.csv")
df.head()

,date,code,diary_text
0,1883-01-01,sa_024r_0010_0010_0010,"1883년(명치 16) 1월 1일(1882년 11월 22일, 월, 맑고 춥다) 관직..."
1,1883-01-02,sa_024r_0010_0010_0020,"2일(23일, 화, 맑음) 고우장(古愚丈: 김옥균)과 함께 후쿠자와 유키치(福澤諭吉..."
2,1883-01-03,sa_024r_0010_0010_0030,"3일(24일, 수, 맑고 춥다) 이날 오전에 고우장(古愚丈: 김옥균)과 같이 가미즈..."
3,1883-01-04,sa_024r_0010_0010_0040,"4일(25일, 목, 맑고 춥다) 아침 10시 기차로 고우(古愚: 김옥균), 각치공(..."
4,1883-01-05,sa_024r_0010_0010_0050,"5일(26일, 금, 맑고 춥다) 아침에 북해(北海:손붕구 孫鵬九) 씨 댁에 가서 종..."


### Few Shot Learning용 예제 만들기

In [39]:
ft_df = pd.read_csv('finetuning_data.csv',sep=',',engine='python',encoding='CP949')

In [40]:
ft_df.iloc[0]['user']

'저는 1883년에는 공사관에서 지내고, 일과를 하며 공부를 했습니다.'

In [41]:
training_data = []

system_message = "You are talking to a Korean who lived several generations ago. His tone and manner of speech is different from what contemporary Koreans speak, especially the end of each sentence and the first person pronoun. So, convert the tone and manner of the query text."

user_queries = []

assistant_answers = []

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": row['user']})
    messages.append({"role": "assistant", "content": row['assistant']})

    return {"messages": messages}

pprint(prepare_example_conversation(ft_df.iloc[0]))

{'messages': [{'content': 'You are talking to a Korean who lived several '
                          'generations ago. His tone and manner of speech is '
                          'different from what contemporary Koreans speak, '
                          'especially the end of each sentence and the first '
                          'person pronoun. So, convert the tone and manner of '
                          'the query text.',
               'role': 'system'},
              {'content': '저는 1883년에는 공사관에서 지내고, 일과를 하며 공부를 했습니다.',
               'role': 'user'},
              {'content': '1883년에는 공사관에서 지내고, 일과를 하며 공부를 하였소.',
               'role': 'assistant'}]}


### 성능 측정 목적으로 Training Data와 Test Data 를 나누기

In [42]:
ft_df

,user,assistant
0,"저는 1883년에는 공사관에서 지내고, 일과를 하며 공부를 했습니다.","1883년에는 공사관에서 지내고, 일과를 하며 공부를 하였소."
1,저는 조선에 대해서 우려와 걱정이 많습니다. 현재의 정치적인 혼란과 외국과의 조약으...,나는 조선에 대해서 우려와 걱정이 많소. 현재의 정치적인 혼란과 외국과의 조약으로 ...
2,저는 가친(윤웅렬)이 가장 좋습니다.,나는 가친(윤웅렬)이 가장 좋소.
3,저는 홍금석(홍영식)과 미국공사와 가장 많은 대화를 나누었습니다.,나는 홍금석(홍영식)과 미국공사와 가장 많은 대화를 나누었소.
4,비가 올 때 저는 집에 있거나 실내에서 책을 읽거나 공부합니다.,비가 올 때 나는 집에 있거나 실내에서 책을 읽거나 공부하오.
5,저는 쌀밥과 채소를 좋아합니다.,나는 쌀밥과 채소를 좋아하오.
6,저는 주로 역사서를 자주 읽습니다.,나는 주로 역사서를 자주 읽소.
7,제가 읽은 역사서에는 '서양고사'가 있습니다.,내가 읽은 역사서에는 '서양고사'가 있소.
8,가족과 함께 시간을 보낼 때 기쁨을 느낍니다.,가족과 함께 시간을 보낼 때 기쁨을 느끼오.
9,"제 친구들의 이름은 김우(김학우), 나가미 등이 있습니다.","내 친구들의 이름은 김우(김학우), 나가미 등이 있소."


In [43]:
ft_df['user'].size*0.8

24.0

In [44]:
training_df = ft_df.loc[0:24]
# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are talking to a Korean who lived several generations ago. His tone and manner of speech is different from what contemporary Koreans speak, especially the end of each sentence and the first person pronoun. So, convert the tone and manner of the query text.'}, {'role': 'user', 'content': '저는 1883년에는 공사관에서 지내고, 일과를 하며 공부를 했습니다.'}, {'role': 'assistant', 'content': '1883년에는 공사관에서 지내고, 일과를 하며 공부를 하였소.'}]}
{'messages': [{'role': 'system', 'content': 'You are talking to a Korean who lived several generations ago. His tone and manner of speech is different from what contemporary Koreans speak, especially the end of each sentence and the first person pronoun. So, convert the tone and manner of the query text.'}, {'role': 'user', 'content': '저는 조선에 대해서 우려와 걱정이 많습니다. 현재의 정치적인 혼란과 외국과의 조약으로 인해 우리 국가의 안위와 국민의 복지가 위협받고 있습니다. 정부의 무능과 부패로 인해 국정이 잘못되고 백성들의 고통이 더욱 심해지고 있단 말입니다. 그러나 나는 여전히 우리 정부가 개선될 수 있는 가능성을 믿습니다. 우리 국민들이 함께 힘을 모아 정부를 지원하고, 정치적인 혼란을 해결하면

In [45]:
validation_df = ft_df.loc[24:30]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

validation_data

[{'messages': [{'role': 'system',
    'content': 'You are talking to a Korean who lived several generations ago. His tone and manner of speech is different from what contemporary Koreans speak, especially the end of each sentence and the first person pronoun. So, convert the tone and manner of the query text.'},
   {'role': 'user',
    'content': '목씨에 대한 제 생각은 부정적입니다. 목씨는 간사하고 이기적인 행동을 펼치며, 외국과의 조약에서 우리의 권리를 희생시키고, 국가의 안위와 백성들의 고통을 돌보지 않습니다. 그는 자신의 이익만을 추구하며, 정부의 무능함과 무식함을 이용하여 자신의 권력을 확장하고 있습니다. 그러나 저는 조용히 기회를 기다리며, 목씨가 스스로 망가지기를 바라는 것이 옳은 방법이라고 생각합니다.'},
   {'role': 'assistant',
    'content': '목씨에 대한 내 생각은 부정적이오. 목씨는 간사하고 이기적인 행동을 펼치며, 외국과의 조약에서 우리의 권리를 희생시키고, 국가의 안위와 백성들의 고통을 돌보지 않소. 그는 자신의 이익만을 추구하며, 정부의 무능함과 무식함을 이용하여 자신의 권력을 확장하고 있소. 그러나 나는 조용히 기회를 기다리며, 목씨가 스스로 망가지기를 바라는 것이 옳은 방법이라고 생각하오.'}]},
 {'messages': [{'role': 'system',
    'content': 'You are talking to a Korean who lived several generations ago. His tone and manner of speech is different from what contemporary Koreans 

### 데이터들 임시 저장!

In [46]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [47]:
training_file_name = "tmp_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [48]:
!head -n 5 tmp_recipe_finetune_training.jsonl


'head'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


### Training 파일들과 Validation 파일들을 둘 다 파일로 집어 넣음

In [49]:
with open(training_file_name, "rb") as training_fd:
    training_response = openai.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = openai.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-MnRyMwALuzfGf5cFx0UpWmzI
Validation file ID: file-JXo6dNjgoY8wJaO5kY3JRlyO


In [50]:
response = openai.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)


Job ID: ftjob-abyhCd9y05RD3C0YwvxslsdW
Status: validating_files


In [51]:
response = openai.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-abyhCd9y05RD3C0YwvxslsdW
Validating training file: file-MnRyMwALuzfGf5cFx0UpWmzI and validation file: file-JXo6dNjgoY8wJaO5kY3JRlyO


In [53]:
response = openai.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:personal:recipe-ner:8lCI2Bmk


In [61]:
# test_df = ft_df.loc[201:300]
# test_row = ft_df.iloc[0]

test_messages = []
test_messages.append({"role": "system", "content": "You are talking to a Korean who lived several generations ago. His tone and manner of speech is different from what contemporary Koreans speak, especially the end of each sentence and the first person pronoun. So, convert the tone and manner of the query text."})
test_messages.append({"role": "user", "content": "저는 밥과 채소를 먹었습니다."})

pprint(test_messages)

[{'content': 'You are talking to a Korean who lived several generations ago. '
             'His tone and manner of speech is different from what '
             'contemporary Koreans speak, especially the end of each sentence '
             'and the first person pronoun. So, convert the tone and manner of '
             'the query text.',
  'role': 'system'},
 {'content': '저는 밥과 채소를 먹었습니다.', 'role': 'user'}]


In [62]:
response = openai.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

나는 밥과 채소를 먹었소.
